In [22]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

### Tool Creation :

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/api_key/pair/{base_currency}/{target_currency}'  # Put your API key in place of api_key

  response = requests.get(url)

  return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [24]:
print(convert.args,convert.description,convert.name,sep='\n')

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'}, 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}
given a currency conversion rate this function calculates the target currency value from a given base currency value
convert


In [25]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1759536002,
 'time_last_update_utc': 'Sat, 04 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1759622402,
 'time_next_update_utc': 'Sun, 05 Oct 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.8022}

In [26]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

### Tool Binding :

In [27]:
# tool binding
llm = ChatGroq(model='llama-3.1-8b-instant')

In [28]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

### Tool Calling :

In [29]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [30]:
ai_message = llm_with_tools.invoke(messages)

In [31]:
messages.append(ai_message)
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nn96g91x5', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'gd2060jcs', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":0.014}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 350, 'total_tokens': 397, 'completion_time': 0.083745628, 'prompt_time': 0.020468361, 'queue_time': 0.069276732, 'total_time': 0.104213989}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7b306ef3-3f4c-423f-a101-27af47670679-0', tool_calls=[{'name': 'get_conversion_factor', 'args':

In [32]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'nn96g91x5',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_factor': 0.014},
  'id': 'gd2060jcs',
  'type': 'tool_call'}]

### Tool Execution :

In [33]:
import json

for tool_call in ai_message.tool_calls:

  # Execute the 1st tool & get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)   # Fetch this conversion rate

    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1) # Append this tool message to messages list

  # Execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':

    tool_call['args']['conversion_rate'] = conversion_rate  # Fetch the current arg

    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)


In [34]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nn96g91x5', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'gd2060jcs', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":0.014}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 350, 'total_tokens': 397, 'completion_time': 0.083745628, 'prompt_time': 0.020468361, 'queue_time': 0.069276732, 'total_time': 0.104213989}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7b306ef3-3f4c-423f-a101-27af47670679-0', tool_calls=[{'name': 'get_conversion_factor', 'args':

In [35]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 1 USD = 88.6637 INR. Based on this conversion factor, 10 INR is approximately 0.1126 USD.'

In [36]:
# from langchain.agents import initialize_agent, AgentType

# # Step 5: Initialize the Agent ---
# agent_executor = initialize_agent(
#     tools=[get_conversion_factor, convert],
#     llm=llm,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
#     verbose=True  # shows internal thinking
# )

In [37]:
# # --- Step 6: Run the Agent ---
# user_query = "Hi how are you?"

# response = agent_executor.invoke({"input": user_query})